In [1]:
%cd ..

/volatile/home/Zaccharie/workspace/Deep-MRI-Reconstruction


In [2]:
from cascadenet_pytorch.model_pytorch import DnCn
from data_torch import MaskedUntouched2DDataset, MaskedUntouched2DAllLoadedDataset
from torch_training import fit_torch, torch_psnr

ModuleNotFoundError: No module named 'cascade_net_pytorch'

In [3]:
# paths
train_path = '/media/Zaccharie/UHRes/singlecoil_train/singlecoil_train/'
val_path = '/media/Zaccharie/UHRes/singlecoil_val/'
test_path = '/media/Zaccharie/UHRes/singlecoil_test/'

n_samples_train = 34742
n_samples_val = 7135

n_volumes_train = 973
n_volumes_val = 199

In [4]:
# data loader
# generators
AF = 4
# train_gen = MaskedUntouched2DAllLoadedDataset(train_path, af=AF, inner_slices=1)
train_gen = MaskedUntouched2DDataset(train_path, af=AF, inner_slices=1)
val_gen = MaskedUntouched2DDataset(val_path, af=AF)
train_loader = DataLoader(
    dataset=train_gen,
    batch_size=1,
    shuffle=False,
    num_workers=35,
    pin_memory=True,
)
val_loader = DataLoader(
    dataset=val_gen,
    batch_size=1,
    num_workers=35,
    pin_memory=True,
)

NameError: name 'MaskedUntouched2DDataset' is not defined

In [ ]:
run_params = {
    'n_cascade': 5,
    'n_convs': 5,
    'n_filters': 64,
}
n_epochs = 100
run_id = f'cascadenet_orig_torch_af{AF}_{int(time.time())}'
chkpt_path = f'checkpoints/{run_id}' + '-{epoch:02d}.hdf5'
log_dir = op.join('logs', run_id)
print(run_id)

In [ ]:
model = CascadeNet(**run_params)
optimizer = Adam(model.parameters(), lr=1e-3)
writer = SummaryWriter(log_dir=log_dir)

model.cuda();

In [ ]:
def overfit_epoch(model, data, optimizer, device):
    model.train()
    kspace, mask, image_gt = data
    kspace = kspace[0]
    mask = mask[0]
    image_gt = image_gt[0]
    kspace = kspace.to(device)
    mask = mask.to(device)
    image_gt = image_gt.to(device)
    image_pred = model(kspace, mask)


    loss = F.l1_loss(image_pred, image_gt)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    psnr = torch_psnr(image_pred, image_gt)
    print('Training PSNR:', psnr)

In [ ]:
%%time
i, data = next(enumerate(train_loader))
for _ in tqdm_notebook(range(500)):
    overfit_epoch(model, data, optimizer, 'cuda')